# Details of the Experiment Interface

The experiment class provides an interface that you can manage your experiment with backward compatibility. It means that even your Experiment has been built/defined you will be able to configure its parameters. This feature will provide more control over your experiment even after your running your experiment for several rounds. In this tutorial, detailed experiment interface will be explained using MNIST basic example.

## Configuring Environment
Before running this notebook, you need to configure your environment by completing following steps:

### Starting the Network Component
Please run following command to start Network component that provided communication between your notebook and the node;
```shell
{FEDBIOMED_DIR}/scripts/fedbiomed_run network
```
<div class="note">
<p>This command will launch docker containers. Therefore, please make sure that your Docker engine is up and running.</p>
</div>

### Deploying MNIST Dataset in the None
Please run following command to add MNIST dataset into your Node. This command will deploy MNIST dataset in your default node whose config file is located in `{FEDBIOMED_DIR}/etc` directory as `config_node.ini`

After running following command, please select data type `2) default`, use default `tags` and select the folder where MNIST dataset will be saved.

```shell
{FEDBIOMED_DIR}/scripts/fedbiomed_run node add
```

### Starting the Node
 After you have successfully completed previous step, please run following commad to start your node.

```shell
{FEDBIOMED_DIR}/scripts/fedbiomed_run node start
```

## Creating a Model

Before declaring an experiment, the model that will be used for federated training should be defined. The model that is goıng to be used is exactly the same model that has been created in the Basic MNIST tutorial. We recommend you to follow Basic MNIST tutorial on PyTorch Framework to understand follwing steps. s

In [34]:
import os
import tempfile
from fedbiomed.researcher.environ import environ

tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR'])
model_file = os.path.join(tmp_dir_model.name, 'class_export_mnist.py')

In [ ]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyTrainingPlan, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
               "from torch.utils.data import DataLoader"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


After runing the cells above, your model codes will be saved in path which is defined in the variable `model_file`. This path will be used let while declaring an experiment.  

## Craeting an Experiment Step by Step  

The experiment class can be created without passing any argument. This will just build an empty experiment object. Afterwards, you will be able define your arguments using setter methods.  


<div class="note"><p>It is always possible to create a fully configured experiment by passing all arguments during the intialization. You can also create your experiment with some of the arguments and set the other arguments after.</p></div>

### Building an Empty Experiment


After building an empty experiment you won't be able to perform federated training, since it is not fully configured. That's why the output of the initialization will always remind you that the experiment experiment is not fully configured by logging required arguments.   

In [3]:
from fedbiomed.researcher.experiment import Experiment
exp = Experiment()

2022-02-18 10:44:34,804 fedbiomed DEBUG - Experiment not fully configured yet: no training data
2022-02-18 10:44:34,805 fedbiomed DEBUG - Experiment not fully configured yet: no node selection strategy
2022-02-18 10:44:34,806 fedbiomed DEBUG - Experiment not fully configured yet: no valid model, model_class=None model_path=None
2022-02-18 10:44:34,807 fedbiomed DEBUG - Experiment not fully configured yet: no job. Missing proper model definition (model_class=None model_path=None)


### Displaying Current Status of Experiment
As an addition to output of the initialization, to find out more about the currrent status of the experiment, you can call the `info()` method of experiment object. This method will print the information about your experiment and what you should complete to be able to start your federated training. 

In [4]:
exp.info()

Arguments           Values
------------------  ----------------------------------------
Tags                None
Nodes filter        None
Training Data       None
Aggregator          <fedbiomed.researcher.aggregators.fedavg
                    .FedAverage object at 0x1036f4070>
Strategy            None
Job                 None
Model Path          None
Model Class         None
Model Arguments     {}
Training Arguments  {}
Rounds already run  0
Rounds total        None
Experiment folder   Experiment_0001
Experiment Path     /Users/sergencansiz/Documents/Inria/Fed-
                    BioMed/fedbiomed/var/experiments/Experim
                    ent_0001
Breakpoint State    False
Monitoring          None

Experiment cannot be run (not fully defined), missing :
- Training Data
- Strategy
- Model
- Job



Based on the output, some arguments are defined with default values, while others are not. Model arguments, training arguments, tags, round limit, training data etc. have no default value and they are required to be set. However, these arguments are related to each other. For example, to be able to define your federeated training data you need to define the `tags` first, and then while setting your training data argument, experiment will be able to send search request to the nodes to recive information about the datasets. These relation between the arguments will explained in the follwing steps.   

### Setting Model for The Experiment

The model that is going to be used for training can be set in the experiment using the methods `set_model_path` and `set_model_class`. The `model_path` is the path your model is saved as a python script. As you remember, in the previous section, the model class has been created and saved in the path which is defined in the variable `model_file`. The experiment also need to now your class name. You can set your class name as a `string` with `set_model_class`. Since it is a python script (module), class name will be used for importing operation at the back-end. Therefore, it always better to define both argument successively. 

<div class="note">
    <p>
        If you are not running your code in Jupyter notebook (IPython kernel), you directly set your class as it is with <code>set_model_class()</code> as a python class (not string). The experiment will be able to extract source of your class and you won't need to provide the argument <code>model_path</code>. 
    <p/>
</div>

In [39]:
exp.set_model_class(model_class="MyTrainingPlan")
exp.set_model_path(model_path=model_file)

'/Users/sergencansiz/Documents/Inria/Fed-BioMed/fedbiomed/var/tmp/tmp6rlwq56w/class_export_mnist.py'

<div class="note">
    <p>If you set your model path first, setter will log a debug message which will inform you about the model is not defined yet. This si because the model class has been set yet</p>
</div>

### Setting Model and Training Arguments
In the previous step, the model has been defined for experiment. Now, you can define your model arguments and training arguments that will be used respectivly for building your model class and training your model on the node side. The methods `set_model_args` and `set_trainin_args` of experiment will allow you to set these arguments. 

<div class="">
    <p>There is requirement in the order of defening model class and mode/training arguments. It is also possible to 
        define model/training arguments first and model class after. 
    </p>    
<div>


In [40]:
# Model arguments should be an empty Dict, since our model does not require 
# any argument for initialization
model_args = {}

# Training Arguments
training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

exp.set_model_args(model_args=model_args)
exp.set_training_args(training_args=training_args)

{'batch_size': 48,
 'lr': 0.001,
 'epochs': 1,
 'dry_run': False,
 'batch_maxnum': 100}

### Setting Tags

The tags for the dataset search request can be set using `set_tags` method of experiment object. 

<br><div class="note"><p>Setting tags does not mean to send dataset search request. Search request is sent while setting training data. `tags` is the argument that is required for the search request.</p></div>

The arguments tags of `set_tags` method should an array of tags which are in `string` type or just a `string`. 

In [41]:
tags = ['#MNIST', '#dataset']
exp.set_tags(tags = tags)

2022-02-18 12:23:33,556 fedbiomed DEBUG - Experimentation tags changed, you may need to update `training_data`


['#MNIST', '#dataset']

To see the tags that are set, you can run `tags()` method of experiment object. 

In [8]:
exp.tags()

['#MNIST', '#dataset']

### Setting Training Data
Training data is a `FederatedDataset` instance which comes from the module `fedbiomed.researcher.datasets`. There are several ways to define your training data.

1. You can run `set_training_data(training_data=None)` method by passing the argument `training_data` as `None`. This will send search request to the nodes  to get dataset information by using the `tags` which should defined before. 
2. You can provide `training_data` argument which is an instance of `FederatedDataSet`. 
3.  You can provide `training_data` argument as python `dict` and setter will create a `FederatedDataSet` object by it self. 

<div class="note"><p>While using the third option please make sure that your `dict` object is configured as coherent to `FedereatedDataset` schema. Otherwise, you might get error while runing your experiment. </p></div>


In [13]:
training_data = exp.set_training_data(training_data=None)

2022-02-18 11:41:00,292 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-02-18 11:41:00,298 fedbiomed INFO - log from: node_0f7ddd0a-3879-4864-828c-014d8dfce442 / DEBUG - Message received: {'researcher_id': 'researcher_4ad5a2c1-8d01-47d4-91a9-8f7aad804c5e', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-02-18 11:41:10,302 fedbiomed INFO - Node selected for training -> node_0f7ddd0a-3879-4864-828c-014d8dfce442


Since it will send search request to the nodes, the ouput will inform you about selected nodes for training. It means that those nodes are have the dataset and able to train your model.

`set_training_data` will return a `FederatedDataSet` object. You can either use the return value of the setter or the getter for training data which is `training_data()` method of the experiment object. 

In [14]:
training_data = exp.training_data()

To inspect the result in detail you can call the method `data()` of the `FedereatedDataSet` object. This will return a python dictionary thats includes information about the datasets that has been found in the nodes. 

In [15]:
training_data.data()

{'node_0f7ddd0a-3879-4864-828c-014d8dfce442': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28],
   'dataset_id': 'dataset_35c1ef06-8532-45fc-8b8d-343f8d381e0f',
   'dtypes': []}]}

As it mention before, setting training data once doesn't mean that you can change it. You can create a new `FederatedDataSet` with a `dict` that includes the information about datasets. This will allow you to select the datasets that will be used for federeated training. 

<div class="note"><p>Since the dataset information will be provided, there will be no need to send request to the nodes</p></div>

In [20]:
from fedbiomed.researcher.datasets import FederatedDataSet 

tr_data = training_data.data()
federeated_dataset = FederatedDataSet(tr_data)
exp.set_training_data(training_data = federeated_dataset)

Or, you can directly use `tr_data` in `set_training_data()`

In [21]:
exp.set_training_data(training_data = tr_data)

<div class="name">
    <p>
        If you change the tags for the dataset by using <code>set_tags</code> and if there is already a defined trainign data in your experiment object, you have to update your training data by running <code>exp.set_training_data(training_data=None)</code>.  
    </p>
</div>

### Setting an Aggregator  

An aggreagtor is one of the required arguments for the experiment. It is used for aggergating model parameters that are received from the nodes after every round. By default, when the experiment is initialized without passing any aggregator, it will auotmaticly use the default `FedAverage` aggregator class. However, it also possbile to set different aggregation algorithm with the method `set_aggregator`. Currently, Fed-BioMed has only `FedAvereage` but it is possible to create custom aggregator classes.

You can see the dcurrent aggregator by running `exp.aggregator()`. It will return the aggregator object that will be used for aggregation. 

In [43]:
exp.aggregator()


If we supposed that you have created your own aggergator, you can set that aggrator as follows, 

In [49]:
from fedbiomed.researcher.aggregators.fedavg import FedAverage
exp.set_aggregator(aggregator=FedAverage)

You can also build your class and pass as an object in case of your aggregator class needs initialization paramters.

In [ ]:
fed_avereage = FedAverage()
exp.set_aggregator(aggregator=fed_avereage)

### Setting Node Selection Strategy

Node selection Strategy is also one of the required arguments for the experiment. It is used for selecting nodes before each round of training. Since the strategy will be used for selecting nodes, before setting strategy, training data should be already set. Then, stragety will be able to which nodes are current with their dataset. 

By default, `set_strategy(node_selection_strategy=None)` will use the default `DefaultStrategy` class. It is default strategy that selects all nodes avaible with their datasets at the moment. However, it also possbile to set different strategies. Currently, Fed-BioMed has only `DefaultStrategy` but it is possible to create custom strategy classes.



In [54]:
exp.set_strategy(node_selection_strategy=None)

Or, you directly pass `DefaultStrategy`

In [56]:
from fedbiomed.researcher.strategies.default_strategy import DefaultStrategy
exp.set_strategy(node_selection_strategy=DefaultStrategy)

# To make the strategy has been set
exp.strategy()

### Setting Round Limit

Round limit is the limit that indicates max number of rounds of the training. By default it is `None` and it needs to be set before running your experiment. You can set the round limit with the method `set_round_limit`. Round limit can  be changed after runing several ruounds of training. You can always excute `exp.round_limit()` to see current round limit. 

In [58]:
exp.set_round_limit(round_limit=2)
exp.round_limit()

2

### Setting Job to Manage Federeated Training Rounds

Job is a class that manages federeated training rounds. Before setting job, stragety for selecting nodes, model and training data should be set. Therefore, please make sure that they all defined before setting job.  The method `set_job` creates the Job instance and it does not take any argument. 

In [59]:
exp.set_job()
exp.job()

2022-02-18 13:14:42,625 fedbiomed DEBUG - torchnn saved model filename: /Users/sergencansiz/Documents/Inria/Fed-BioMed/fedbiomed/var/experiments/Experiment_0001/my_model_5018329c-fae2-4f09-9a71-952e0d165f42.py


### Last Check
Now, let's if our experiment is ready for the training by running `exp.info()`. 

In [60]:
exp.info()

Arguments           Values
------------------  ----------------------------------------
Tags                ['#MNIST', '#dataset']
Nodes filter        None
Training Data       <fedbiomed.researcher.datasets.Federated
                    DataSet object at 0x1349d9460>
Aggregator          <fedbiomed.researcher.aggregators.fedavg
                    .FedAverage object at 0x134975790>
Strategy            <fedbiomed.researcher.strategies.default
                    _strategy.DefaultStrategy object at 0x13
                    49d9af0>
Job                 <fedbiomed.researcher.job.Job object at
                    0x134a926a0>
Model Path          /Users/sergencansiz/Documents/Inria/Fed-
                    BioMed/fedbiomed/var/tmp/tmp6rlwq56w/cla
                    ss_export_mnist.py
Model Class         MyTrainingPlan
Model Arguments     {}
Training Arguments  {'batch_size': 48, 'lr': 0.001, 'epochs'
                    : 1, 'dry_run': False, 'batch_maxnum': 1
                    00}
Rounds 

If the experiment is ready, you will see the message that says `Experiment can be run now (fully defined)` at the bottom of the output. So now, we can run the experiment

## Runing The Experiment

In [57]:
exp.run()

2022-02-18 13:05:29,684 fedbiomed WARNING - Cannot run, please specify a number of `rounds` to run or set a `round_limit` to the experiment


0

In [ ]:
exp.run_once()

In [ ]:
exp.run(1)

In [ ]:
exp.run()

In [ ]:
exp.run_once()
exp.run()

In [ ]:
exp.set_rounds(4)

In [ ]:
exp.run()

In [ ]:
exp.run_once(True)

In [ ]:
exp.run(1, True)

In [ ]:
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Index                          : ' , list(range(exp.round_current())))

## Declaring an Experiment Step by Step 
### Building Empty Experiment

In [ ]:
tags =  ['#MNIST', '#dataset']
from fedbiomed.researcher.requests import Requests
reqs = Requests()
training_data = reqs.search(tags)

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.strategies.default_strategy import DefaultStrategy
#strategy= DefaultStrategy(training_data)
from fedbiomed.researcher.aggregators.fedavg import FedAverage
strategy = FedAverage()
exp = Experiment(training_data=training_data, node_selection_strategy=strategy)

In [ ]:
from fedbiomed.researcher.experiment import Experiment
exp = Experiment()

### Setting Tags 

Tags should list strings that contains tags or a string with single tag. 

---
<div class="note">
    <p>If provided tags is not in correct type `.set_tags` will raise <code>TypeError</code></p>
</div>

In [ ]:
tags = ["#MNIST", "#dataset"]
exp.set_tags(tags = tags)

### Setting Model Path and Model Model Class

In [ ]:
exp.set_model_path(model_path = model_file)
exp.set_model_class(model_class = 'MyTrainingPlan')

### Setting Model Arguments and Training Arguments

In [ ]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100
}

exp.set_model_args(model_args = model_args)
exp.set_training_args(training_args = training_args)

### Setting Training Data

The method `set_trainig_data` gets there arguments: 

- `tags` : List of tags as string for the search request. If it is not provided. The method will try to use `tags` attribute of the object. 
- `nodes`: List of node ids that a search request will be sent. If this argument is not provided search request will be sent to all active nodes.  
- `training_data`: A dictionary or `FederatedDataset` object. If `training_data` provided search request with `tags` and `nodes` will be ignored.

In [ ]:
exp.set_training_data()


### Setting Job 

Setting job will prepare all neccessary assets to be able to run a round. Therefore, `Job` should be set before running the experiment.  

To be able to set `Job`, you should be already set the arguments: `model_path`, `model_class`, `training_data`. Otherwiser `set_job()` will reaise an Exception. 

In [ ]:
exp.set_job()

In [ ]:
exp.set_node_selection_strategy()

Parameters of The Experiment

In [ ]:
print('Rounds              :', exp.rounds())
print('Tags                :', exp.tags())
print('Model Path          :', exp.model_path())
print('Model Class         :', exp.model_class())
print('Model Arguments     :', exp.model_args())
print('Training Arguments  :', exp.training_args())
print('Job                 :', exp.job())
print('Training Data       :', exp.training_data())
print('Job                 :', exp.job())
print('Nodes               :', exp.nodes()) # Returns selected nodes after search request
print('Aggregator          :', exp.aggregator())
print('N.S. Stragety       :', exp.node_selection_strategy())
print('Breakpoint State    :', exp.breakpoint())
print('Exp  folder         :', exp.experimentation_folder())
print('Exp  path           :', exp.experimentation_path())



In [ ]:
exp.info()

In [ ]:
exp.run_once()

In [ ]:
print('Number of rounds initial             : ' , exp.rounds())
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Indexes                        : ' , list(range(exp.round_current())))

In [ ]:
exp.run_once()

Check current round, deaclare the the round that will be run. 

In [ ]:
print('Number of rounds initial             : ' , exp.rounds())
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Indexes                        : ' , list(range(exp.round_current())))

Running multiple rounds:

In [ ]:
exp.run(rounds=3)

In [ ]:
exp.run_once()

Setting rounds to higher value

In [ ]:
new_rounds = exp.rounds() + 1
exp.set_rounds(new_rounds)
exp.run_once()

In [ ]:
print('Number of rounds initial             : ' , exp.rounds())
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Indexes                        : ' , list(range(exp.round_current())))

In [ ]:
rounds = exp.round_current()

print("\nList the training rounds : ", exp.training_replies().keys())
print("\nList the nodes for the last training round and their timings : ")
for r in exp.training_replies().keys():
    round_data = exp.training_replies()[r].data()
    print('\n\t Round %s' % str(r+1))
    for c in range(len(round_data)):
        print("\t\t- {id} :\
        \n\t\t\trtime_training={rtraining:.2f} seconds\
        \n\t\t\tptime_training={ptraining:.2f} seconds\
        \n\t\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
            rtraining = round_data[c]['timing']['rtime_training'],
            ptraining = round_data[c]['timing']['ptime_training'],
            rtotal = round_data[c]['timing']['rtime_total']))
print('\n')

### Run Same Experiment with Multple Rounds

In [ ]:
exp.run(rounds=2)

### Changing Experiment Parameters with Setters after all The Argument is Already Set
If the `Job` is already initialize and the arguments related to model is modified, `Job` should reinitialize with the method `.set_job()`. This information is also given by Experiment after setting model file.  
  
    
    
<div class="note">
    <p>After runing the experiment changing the model might have some consequances.</p>
</div>

In [ ]:
exp.set_model_path(model_file)
exp.set_model_class('MyTrainingPlan')

In [ ]:
exp.set_job()

#### Changing Aggregator

Aggregator should be instance of `fedbiomed.researcher.aggregators.aggregator.Aggregator`. Otherwise `set_aggregator` will raise an Expection. Aggregator should be passed as `Callable` class or alredy built object.

Following cell will raise an Exception:

In [ ]:
exp.set_aggregator('ThisIsNotAnAggregator')

Correct usage: 

In [ ]:
from fedbiomed.researcher.aggregators.fedavg import FedAverage
# Can be passed as Callable class
exp.set_aggregator(FedAverage)

# Can be passed as already build class
fedavg = FedAverage()
exp.set_aggregator(fedavg)

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D